<a href="https://colab.research.google.com/github/saspav/DA-104/blob/main/%D0%9F%D0%B0%D0%B2%D0%BB%D0%BE%D0%B2%D0%B0_%D0%A1%D0%92_%D0%9F%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D1%81_%D1%81%D0%B0%D0%B9%D1%82%D0%BE%D0%B2_%D0%BD%D0%B0_Python_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_%D0%B1%D0%B8%D0%B1%D0%BB%D0%B8%D0%BE%D1%82%D0%B5%D0%BA%D0%B8_BeautifulSoup_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install random_user_agent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Задача 1

Написать код, который соберет список знаменательных дат в формате "чисто месяц год" с первой страницы сайта GCTC.ru (http://www.gctc.ru/main.php?id=98.1)

In [ ]:
import re
import pandas as pd
import requests
from time import sleep
from random import random
from datetime import datetime
from bs4 import BeautifulSoup
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem

__import__('warnings').filterwarnings("ignore")


soft_names = [SoftwareName.CHROME.value]
op_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]
user_agent = UserAgent(software_names=soft_names, operating_systems=op_systems, limit=99)

In [ ]:
def get_html(page):
    """Собирает html"""
    # Инициализация сессии
    session = requests.session()
    # Указание заголовков сессии
    session.headers = {'User-Agent': user_agent.get_random_user_agent()}
    try:
        # Указание заголовков сессии
        res = session.get(URL.format(page))
        # Проверка на ошибку
        res.raise_for_status()
    except Exception as ex:
        print(ex)
        return

    return res.text


def get_soup(page):
    """ Варим суп """
    # Получаем исходный код
    html = get_html(page)
    # Если нет страницы, то выходим
    if not html:
        return
    # Начинаем варить суп (Инициализируем bs)
    soup = BeautifulSoup(html, "lxml")
    return soup

In [ ]:
URL = "https://www.gctc.ru/main.php?id=98.{}"

def get_page_info1(page, soup):
    """ Собирает информацию со страницы"""
    ie_infoh = soup.select_one('.ie_infoh')
    day = ''
    df = pd.DataFrame(columns=['date', 'event'])
    for child in ie_infoh.contents:
        if child:
            data = BeautifulSoup(str(child), "lxml")
            tag_h1 = data.select('h1')
            news = data.select('.news')
            if tag_h1:
                elems = list(tag_h1[0].stripped_strings)
                day = elems[0]
            elif news:
                elems = list(news[0].stripped_strings)
                year, _, event = elems
                df.loc[len(df)] = [datetime(int(year), page, int(day)), event]
    return df

# Написать код, который соберет список знаменательных дат в формате
# "чисто месяц год" с первой страницы сайта GCTC.ru
result = pd.DataFrame(columns=['date', 'event'])
# Цикл по количеству страниц
for page in range(1, 13):
    print(f'Месяц: {page:02}')
    soup = get_soup(page)
    if not soup:
        # Если нет супа, то выходим из цикла => либо ошибка, либо конец страниц
        break
    # Получаем информацию
    page_info = get_page_info1(page, soup)
    # Записываем в result инфу со страницы
    result = pd.concat([result, page_info])
    # Перерыв между запросами
    sleep(3 * random())

Месяц: 01
Месяц: 02
Месяц: 03
Месяц: 04
Месяц: 05
Месяц: 06
Месяц: 07
Месяц: 08
Месяц: 09
Месяц: 10
Месяц: 11
Месяц: 12


In [ ]:
# преобразование даты в нужный формат
result.date = result.date.dt.strftime('%d.%m.%Y')
# сохранение в .CSV
result.to_csv('Знаменательные_даты.csv', index=False)

In [ ]:
# просмотр результатов
result

,date,event
0,01.01.1942,Сарафанов Геннадий Васильевич родился 1 января...
1,01.01.1947,Титов Владимир Георгиевич родился 1 января 194...
2,01.01.1956,Авдеев Сергей Васильевич родился 1 января 1956...
3,01.01.1967,НИИ-88 преобразован в Центральный НИИ Машиност...
4,02.01.1985,Принято постановление правительства о создании...
...,...,...
14,24.12.1969,Скрипочка Олег Иванович родился 24 декабря 19...
15,26.12.1974,Запуск орбитальной станции «Салют-4». В полете...
16,28.12.1970,Олег Германович Артемьев родился 28 декабря 19...
17,29.12.1974,Евгений Игоревич Тарелкин родился 29 декабря 1...


# Задача 2

Соберите информацию о  происходивших выставках кошек с сайта RU-pets.ru <br>(http://ru-pets.ru/index.php?m=6&c=2&to=1).

Данные, которые необходимы:
* Дата проведения
* Название выставки
* Клуб-Организатор
<br>

Результат необходимо записать в CSV файл

In [ ]:
URL = "http://ru-pets.ru/index.php?m=6&to=1&c=2&page={}"

def get_page_info2(soup):
    """ Собирает информацию со страницы"""
    df = pd.DataFrame(columns=['date', 'name', 'club'])
    for item in soup.select('.listitem'):
        period = list(item.select_one('h2').stripped_strings)
        if period:
            date = period[0].split(',')[0]
            name = period[1]
            club = ''
            info = list(item.select_one('.msgtext').stripped_strings)
            if info:
                for elem in ''.join(info).split(';'):
                    if re.findall('клуб.*организатор', elem.lower()):
                        club = elem.split(':')[-1]
                        break
            df.loc[len(df)] = [date, name, club]
    return df


result = pd.DataFrame(columns=['date', 'name', 'club'])

# получаем количество страниц
total_pages = int(get_soup(1).select('#page')[-1].contents[0])
# Цикл по количеству страниц
print('Страницы:', end=' ')
for page in range(1, total_pages + 1):
    print(page, end=' ')
    if not page % 20:
        print()
    soup = get_soup(page)
    if not soup:
        # Если нет супа, то выходим из цикла => либо ошибка, либо конец страниц
        break
    # Получаем информацию
    page_info = get_page_info2(soup)
    # Записываем в result инфу со страницы
    result = pd.concat([result, page_info])
    # Перерыв между запросами
    sleep(3 * random())

result.columns = ['Даты проведения', 'Название выставки', 'Клуб-Организатор']
result.to_csv('выставки_кошек.csv', index=False)

Страницы: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 
41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 
61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 

In [ ]:
# просмотр результатов
result

,Даты проведения,Название выставки,Клуб-Организатор
0,12-13 июня 2022г.,«ЛЕТО - 2022»,Самарис
1,23 октября 2021г.,«КЭТ-САЛОН-ОКТЯБРЬ»,КЛК Москва
2,9-10 октября 2021г.,«РОСКОШНАЯ ОСЕНЬ»,КЛК РосКош
3,2-3 октября 2021г.,«ОСЕННИЙ ВЕРНИСАЖ-2021»,УРОФО Грация
4,2-3 октября 2021г.,«КОТОМИР ШОУ»,НП КЛК Котомир
...,...,...,...
4,25 сентября 2010г.,«XII Международный фестиваль «ЭКСПОКОТ 2010»»,
5,25 сентября 2010г.,«Уральский ГлаМУРР»,
6,25 сентября 2010г.,«Котомир-Шоу – 2010»,
7,25 сентября 2010г.,«Невский Талисман»,
